# Knative

Knative (ausgesprochen kay-nay-tiv) erweitert Kubernetes um eine Reihe von Middleware-Komponenten, die für die Erstellung moderner, container-basierter Anwendungen unerlässlich sind.

Knative Komponenten konzentrieren sich auf die Lösung alltäglicher, aber schwieriger Aufgaben wie:

* Bereitstellen eines Containers
* Routing und Verwaltung des Datenverkehrs mit blue / green Bereitstellung
* Automatische Skalierung des Workloads
* Bindung laufender Dienste an mehrere Ökosysteme

***
## Serving

![](https://github.com/knative/serving/raw/main/docs/spec/images/object_model.png)

Quelle: [knative.dev](https://knative.dev/docs/serving/#serving-resources)
- - -

Knative Serving baut auf Kubernetes und Istio auf, um die Bereitstellung von Anwendungen und Funktionen ohne Server (Serverless, Function as a Service FaaS) zu unterstützen. 

Das Knative Serving-Projekt bietet Middleware-Grundelemente, die Folgendes ermöglichen:

* Schnelle Bereitstellung von Services ohne Server
* Automatische Skalierung auf 0
* Routing und Netzwerkeinstellungen für Istio-Komponenten
* Momentaufnahmen von bereitgestelltem Code und bereitgestellten Konfigurationen

### Installation

* [Installing Knative Serving using YAML files](https://knative.dev/docs/install/yaml-install/serving/install-serving-with-yaml/#install-the-knative-serving-component)

In [ ]:
%%bash
kubectl apply -f https://github.com/knative/serving/releases/download/knative-v1.4.0/serving-crds.yaml
kubectl apply -f https://github.com/knative/serving/releases/download/knative-v1.4.0/serving-core.yaml
kubectl apply -l knative.dev/crd-install=true -f https://github.com/knative/net-istio/releases/download/knative-v1.4.0/istio.yaml
kubectl apply -f https://github.com/knative/net-istio/releases/download/knative-v1.4.0/istio.yaml
kubectl apply -f https://github.com/knative/net-istio/releases/download/knative-v1.4.0/net-istio.yaml

Installation überprüfen. Es müssen 8 Container gestartet sein.

In [ ]:
%%bash
kubectl get pods -n knative-serving
kubectl --namespace istio-system get service istio-ingressgateway

***
### Beispiel Serving - Skalierung auf 0

Nehmen wir an wir haben einen Service welcher nur 1 - 2 im Monat verwendet wird.

Um keine Rechnenleistung zu verschwenden, soll dieser Service nur gestartet werden, wenn er wirklich gebraucht wird.

Deshalb wird der Service "Serverless" implementiert.

In [ ]:
! kubectl create namespace serving

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: serving.knative.dev/v1 # Current version of Knative
kind: Service
metadata:
  name: helloworld-go # The name of the app
  namespace: serving # The namespace the app will use
spec:
  template:
    spec:
      containers:
        - image: gcr.io/knative-samples/helloworld-go # The URL to the image of the app
          env:
            - name: TARGET # The environment variable printed out by the sample app
              value: "ich werde nur auf Anforderung gestartet"
%EOF%

Schauen wir uns nun die erstellten Ressourcen an.

Der eigentliche Service (Pods) wird am Anfang gestartet und nach einer bestimmen Leerlaufzeit beendet. Das ReplicaSet setzt die Anzahl Pods auf 0.

In [ ]:
! kubectl get all --namespace serving

Der Aufruf des Services erfolgt via Istio. Dabei ist explizit der Hostname (HTTP Header Host:) mit anzugeben.

Das erste Mal dauert der Aufruf relativ lange (Latenzzeit), weil der Pods zuerst gestartet werden muss.

Alternativ könnten wir den Hosteintrag in die `hosts` Datei eintragen oder einen DNS Server verwenden.

**ACHTUNG**: damit es funktioniert müssen oben alle Ressourcen den Status Ready=True haben. 

In [ ]:
%%bash
date
curl -H "Host: helloworld-go.serving.example.com" istio-ingressgateway.istio-system
date
# zweiter Start
curl -H "Host: helloworld-go.serving.example.com" istio-ingressgateway.istio-system
date

- - -
Aufräumen, kann auch Verwendet werden wenn die Ressoucen auf Status Ready=Unknown verbleiben.

In [ ]:
! kubectl delete namespace serving